<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />

<br><h1>Building a Classification Model in Kaggle</h1>
Sreekar<br>
Hult International Business School<br><br><br>

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" />


Below is a starter code that comes with Jupyter Notebook.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br><br>
You will need to import any additional libraries. It's also a good idea to get rid of the cell above once you're used to Jupyter Notebook in Kaggle.<br>

In [2]:
# modeling library
import sklearn.linear_model                          # linear modeling in scikit-learn

# other model building tools
from sklearn.model_selection import train_test_split # train-test split
from sklearn.metrics import roc_auc_score            # auc score

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br><br>
Make sure to tell Python the path to your training data. If you have no idea what a path is, look to the left and find and hover over <em>Data&nbsp;>&nbsp;Input&nbsp;>&nbsp;spaceship-titanic</em>. A button should appear allowing you to copy the path to your clipboard.<br>

In [3]:
# importing the training dataset
path             = "/kaggle/input/spaceship-titanic/" 
training_dataset = "train.csv"


# reading in the .csv file with pandas
data_train    = pd.read_csv(filepath_or_buffer = path + training_dataset)


# checking basic info about the dataset
data_train.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [4]:
import pandas as pd



# split the "Cabin" column into three separate columns
data_train[['Deck', 'Num', 'Side']] = data_train['Cabin'].str.split('/', expand=True)

# remove the original "Cabin" column
data_train = data_train.drop(columns=['Cabin'])
#data_train = data_train.drop(columns=['PassengerId'])
data_train = data_train.drop(columns=['Name'])


# print the resulting dataframe
data_train.head(2)

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Deck,Num,Side
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False,B,0,P
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True,F,0,S


In [5]:
#most_common_value = data_train['HomePlanet'].mode()[0]              #Text
#data_train['HomePlanet'].fillna(most_common_value, inplace=True)

#most_common_value = data_train['CryoSleep'].value_counts().idxmax() #Bool
#data_train['CryoSleep'].fillna(most_common_value, inplace=True)

#most_common_value = data_train['Deck'].mode()[0]                  
#data_train['Deck'].fillna(most_common_value, inplace=True)

#most_common_value = data_train['Side'].mode()[0]                  
#data_train['Side'].fillna(most_common_value, inplace=True)

#most_common_value = data_train['Destination'].mode()[0]
#data_train['Destination'].fillna(most_common_value, inplace=True)

#data_train['Age'].fillna(data_train['Age'].mean(), inplace=True)

#most_common_value = data_train['VIP'].value_counts().idxmax()
#data_train['VIP'].fillna(most_common_value, inplace=True)

#data_train['RoomService'].fillna(data_train['RoomService'].median(), inplace=True)

#data_train['FoodCourt'].fillna(data_train['FoodCourt'].median(), inplace=True)

#data_train['ShoppingMall'].fillna(data_train['ShoppingMall'].median(), inplace=True)

#data_train['Spa'].fillna(data_train['Spa'].median(), inplace=True)

#data_train['VRDeck'].fillna(data_train['VRDeck'].median(), inplace=True)

#data_train['Num'].fillna(data_train['VRDeck'].median(), inplace=True)

data_train.fillna(method='ffill')

# titanic_train.head(5)
#titanic_train.shape
data_train.info(verbose = True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Destination   8511 non-null   object 
 4   Age           8514 non-null   float64
 5   VIP           8490 non-null   object 
 6   RoomService   8512 non-null   float64
 7   FoodCourt     8510 non-null   float64
 8   ShoppingMall  8485 non-null   float64
 9   Spa           8510 non-null   float64
 10  VRDeck        8505 non-null   float64
 11  Transported   8693 non-null   bool   
 12  Deck          8494 non-null   object 
 13  Num           8494 non-null   object 
 14  Side          8494 non-null   object 
dtypes: bool(1), float64(6), object(8)
memory usage: 959.4+ KB


In [6]:
encoded_data = pd.get_dummies(data_train['HomePlanet'], prefix='HomePlanet',drop_first=True)

# Add the one-hot encoded columns to the original DataFrame
data_train = pd.concat([data_train, encoded_data], axis=1)

In [7]:
encoded_data = pd.get_dummies(data_train['CryoSleep'], prefix='CryoSleep',drop_first=True)

# Add the one-hot encoded columns to the original DataFrame
data_train = pd.concat([data_train, encoded_data], axis=1)

In [8]:
encoded_data = pd.get_dummies(data_train['Destination'], prefix='Destination',drop_first=True)

# Add the one-hot encoded columns to the original DataFrame
data_train = pd.concat([data_train, encoded_data], axis=1)

In [9]:
encoded_data = pd.get_dummies(data_train['VIP'], prefix='VIP',drop_first=True)

# Add the one-hot encoded columns to the original DataFrame
data_train = pd.concat([data_train, encoded_data], axis=1)

In [10]:
encoded_data = pd.get_dummies(data_train['Deck'], prefix='Deck',drop_first=True)

# Add the one-hot encoded columns to the original DataFrame
data_train = pd.concat([data_train, encoded_data], axis=1)

In [11]:
encoded_data = pd.get_dummies(data_train['Side'], prefix='Side',drop_first=True)

# Add the one-hot encoded columns to the original DataFrame
data_train = pd.concat([data_train, encoded_data], axis=1)

In [12]:
# remove the original "Cabin" column
data_train = data_train.drop(columns=['HomePlanet'])
data_train = data_train.drop(columns=['CryoSleep'])
data_train = data_train.drop(columns=['Destination'])
data_train = data_train.drop(columns=['VIP'])
data_train = data_train.drop(columns=['Deck'])
data_train = data_train.drop(columns=['Side'])

In [13]:
from sklearn.preprocessing import LabelEncoder
LE=LabelEncoder()
data_train['Transported']=LE.fit_transform((data_train['Transported']))
data_train.head(2)

,PassengerId,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Num,HomePlanet_Europa,...,Destination_TRAPPIST-1e,VIP_True,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_S
0,0001_01,39.0,0.0,0.0,0.0,0.0,0.0,0,0,1,...,1,0,1,0,0,0,0,0,0,0
1,0002_01,24.0,109.0,9.0,25.0,549.0,44.0,1,0,0,...,1,0,0,0,0,0,1,0,0,1


In [14]:
#from sklearn.preprocessing import PowerTransformer
#PT_yj = PowerTransformer(method='yeo-johnson')

In [15]:
#data_train[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]=PT_yj.fit_transform(data_train[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']])


In [16]:
#from sklearn.preprocessing import StandardScaler
#ss=StandardScaler()
#data_train['Age']=ss.fit_transform(data_train[['Age']])
#data_train['RoomService']=ss.fit_transform(data_train[['RoomService']])
#data_train['FoodCourt']=ss.fit_transform(data_train[['FoodCourt']])
#data_train['ShoppingMall']=ss.fit_transform(data_train[['ShoppingMall']])
#data_train['Spa']=ss.fit_transform(data_train[['Spa']])
#data_train['VRDeck']=ss.fit_transform(data_train[['VRDeck']])

In [17]:
data_train.head(2)

,PassengerId,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,Num,HomePlanet_Europa,...,Destination_TRAPPIST-1e,VIP_True,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_S
0,0001_01,39.0,0.0,0.0,0.0,0.0,0.0,0,0,1,...,1,0,1,0,0,0,0,0,0,0
1,0002_01,24.0,109.0,9.0,25.0,549.0,44.0,1,0,0,...,1,0,0,0,0,0,1,0,0,1


## Test Data

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
The test set is also available. Notice that there is no data for Transported (y-variable) in the test set. This is intentional.<br>

In [18]:
# importing the training dataset
path             = "/kaggle/input/spaceship-titanic/"
testing_dataset  = 'test.csv'

# importing the testing dataset
data_test = pd.read_csv(filepath_or_buffer = path + testing_dataset)

# checking basic info about the dataset
data_test.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [19]:
import pandas as pd



# split the "Cabin" column into three separate columns
data_test[['Deck', 'Num', 'Side']] = data_test['Cabin'].str.split('/', expand=True)

# remove the original "Cabin" column
data_test = data_test.drop(columns=['Cabin'])
#data_test = data_test.drop(columns=['PassengerId'])
data_test = data_test.drop(columns=['Name'])


# print the resulting dataframe
data_test.head(2)

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Deck,Num,Side
0,0013_01,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,G,3,S
1,0018_01,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,F,4,S


In [20]:
#most_common_value = data_test['HomePlanet'].mode()[0]              #Text
#data_test['HomePlanet'].fillna(most_common_value, inplace=True)

#most_common_value = data_test['CryoSleep'].value_counts().idxmax() #Bool
#data_test['CryoSleep'].fillna(most_common_value, inplace=True)

#most_common_value = data_test['Deck'].mode()[0]                  
#data_test['Deck'].fillna(most_common_value, inplace=True)

#most_common_value = data_test['Side'].mode()[0]                  
#data_test['Side'].fillna(most_common_value, inplace=True)

#most_common_value = data_test['Destination'].mode()[0]
#data_test['Destination'].fillna(most_common_value, inplace=True)

#data_test['Age'].fillna(data_test['Age'].mean(), inplace=True)

#most_common_value = data_test['VIP'].value_counts().idxmax()
#data_test['VIP'].fillna(most_common_value, inplace=True)

#data_test['RoomService'].fillna(data_test['RoomService'].median(), inplace=True)

#data_test['FoodCourt'].fillna(data_test['FoodCourt'].median(), inplace=True)

#data_test['ShoppingMall'].fillna(data_test['ShoppingMall'].median(), inplace=True)

#data_test['Spa'].fillna(data_test['Spa'].median(), inplace=True)

#data_test['VRDeck'].fillna(data_test['VRDeck'].median(), inplace=True)

#data_test['Num'].fillna(data_test['VRDeck'].median(), inplace=True)

data_train.fillna(method='ffill')
# titanic_train.head(5)
#titanic_train.shape
data_test.info(verbose = True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Destination   4185 non-null   object 
 4   Age           4186 non-null   float64
 5   VIP           4184 non-null   object 
 6   RoomService   4195 non-null   float64
 7   FoodCourt     4171 non-null   float64
 8   ShoppingMall  4179 non-null   float64
 9   Spa           4176 non-null   float64
 10  VRDeck        4197 non-null   float64
 11  Deck          4177 non-null   object 
 12  Num           4177 non-null   object 
 13  Side          4177 non-null   object 
dtypes: float64(6), object(8)
memory usage: 467.9+ KB


In [21]:
encoded_data = pd.get_dummies(data_test['HomePlanet'], prefix='HomePlanet',drop_first=True)

# Add the one-hot encoded columns to the original DataFrame
data_test = pd.concat([data_test, encoded_data], axis=1)

In [22]:
encoded_data = pd.get_dummies(data_test['CryoSleep'], prefix='CryoSleep',drop_first=True)

# Add the one-hot encoded columns to the original DataFrame
data_test = pd.concat([data_test, encoded_data], axis=1)

In [23]:
encoded_data = pd.get_dummies(data_test['Destination'], prefix='Destination',drop_first=True)

# Add the one-hot encoded columns to the original DataFrame
data_test = pd.concat([data_test, encoded_data], axis=1)

In [24]:
encoded_data = pd.get_dummies(data_test['VIP'], prefix='VIP',drop_first=True)

# Add the one-hot encoded columns to the original DataFrame
data_test = pd.concat([data_test, encoded_data], axis=1)

In [25]:
encoded_data = pd.get_dummies(data_test['Deck'], prefix='Deck',drop_first=True)

# Add the one-hot encoded columns to the original DataFrame
data_test = pd.concat([data_test, encoded_data], axis=1)

In [26]:
encoded_data = pd.get_dummies(data_test['Side'], prefix='Side',drop_first=True)

# Add the one-hot encoded columns to the original DataFrame
data_test = pd.concat([data_test, encoded_data], axis=1)

In [27]:
# remove the original "Cabin" column
data_test = data_test.drop(columns=['HomePlanet'])
data_test = data_test.drop(columns=['CryoSleep'])
data_test = data_test.drop(columns=['Destination'])
data_test = data_test.drop(columns=['VIP'])
data_test = data_test.drop(columns=['Deck'])
data_test = data_test.drop(columns=['Side'])

In [28]:
#from sklearn.preprocessing import PowerTransformer
#PT_yj = PowerTransformer(method='yeo-johnson')

In [29]:
#data_test[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]=PT_yj.fit_transform(data_test[['Age','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']])


In [30]:
#from sklearn.preprocessing import StandardScaler
#ss=StandardScaler()
#data_test['Age']=ss.fit_transform(data_test[['Age']])
#data_test['RoomService']=ss.fit_transform(data_test[['RoomService']])
#data_test['FoodCourt']=ss.fit_transform(data_test[['FoodCourt']])
#data_test['ShoppingMall']=ss.fit_transform(data_test[['ShoppingMall']])
#data_test['Spa']=ss.fit_transform(data_test[['Spa']])
#data_test['VRDeck']=ss.fit_transform(data_test[['VRDeck']])

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
You may want to join the datasets together before feature engineering. This will help make things more efficient.<br>

In [31]:
data_train['set'] = 'Training'
data_test ['set'] = 'Testing'

# concatenating both datasets together for mv and feature engineering
titanic_df = data_train.append(other = data_test)

# resetting index to avoid problems later in the code
titanic_df.reset_index(drop = False,
                       inplace = True)

In [32]:
data_test.head(5)

,PassengerId,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Num,HomePlanet_Europa,HomePlanet_Mars,...,VIP_True,Deck_B,Deck_C,Deck_D,Deck_E,Deck_F,Deck_G,Deck_T,Side_S,set
0,0013_01,27.0,0.0,0.0,0.0,0.0,0.0,3,0,0,...,0,0,0,0,0,0,1,0,1,Testing
1,0018_01,19.0,0.0,9.0,0.0,2823.0,0.0,4,0,0,...,0,0,0,0,0,1,0,0,1,Testing
2,0019_01,31.0,0.0,0.0,0.0,0.0,0.0,0,1,0,...,0,0,1,0,0,0,0,0,1,Testing
3,0021_01,38.0,0.0,6652.0,0.0,181.0,585.0,1,1,0,...,0,0,1,0,0,0,0,0,1,Testing
4,0023_01,20.0,10.0,0.0,635.0,0.0,0.0,5,0,0,...,0,0,0,0,0,1,0,0,1,Testing


In [33]:
titanic_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   index                      12970 non-null  int64  
 1   PassengerId                12970 non-null  object 
 2   Age                        12700 non-null  float64
 3   RoomService                12707 non-null  float64
 4   FoodCourt                  12681 non-null  float64
 5   ShoppingMall               12664 non-null  float64
 6   Spa                        12686 non-null  float64
 7   VRDeck                     12702 non-null  float64
 8   Transported                8693 non-null   float64
 9   Num                        12671 non-null  object 
 10  HomePlanet_Europa          12970 non-null  uint8  
 11  HomePlanet_Mars            12970 non-null  uint8  
 12  CryoSleep_True             12970 non-null  uint8  
 13  Destination_PSO J318.5-22  12970 non-null  uin

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Let's look at correlations to find an x-variable to model with.<br>

In [34]:
# instantiating a correlation matrix
titanic_corr = data_train.corr(method = 'pearson').round(decimals = 2)

# transforming correlations to absolute values
titanic_corr.loc[ : , 'Transported' ].apply(func = abs).sort_values(ascending = False)

Transported                  1.00
CryoSleep_True               0.46
RoomService                  0.24
Spa                          0.22
VRDeck                       0.21
HomePlanet_Europa            0.18
Deck_B                       0.14
Deck_C                       0.11
Side_S                       0.10
Deck_E                       0.10
Deck_F                       0.09
Destination_TRAPPIST-1e      0.09
Age                          0.08
FoodCourt                    0.05
VIP_True                     0.04
Deck_D                       0.03
HomePlanet_Mars              0.02
Deck_G                       0.02
ShoppingMall                 0.01
Deck_T                       0.01
Destination_PSO J318.5-22    0.00
Name: Transported, dtype: float64

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Since training and testing sets have already been developed for us, we can simply select the X-features we want to model with and we're good to go.<br>

In [35]:

# imputing in missing values for RoomService
#titanic_df[ 'RoomService' ].fillna(value = 0, inplace = True)

# setting explanatory variable(s) with most correlated x-variable
x_train = titanic_df[ ['CryoSleep_True','Spa','RoomService','VRDeck','ShoppingMall','FoodCourt','HomePlanet_Europa','Deck_B','Deck_C','Side_S','Destination_TRAPPIST-1e','Deck_E','Deck_F','Age','VIP_True','Deck_D','HomePlanet_Mars','Deck_G','Deck_T','Destination_PSO J318.5-22'] ][ titanic_df['set'] == 'Training' ]
#x_train = titanic_df[ ['CryoSleep_True','Spa','RoomService'] ][ titanic_df['set'] == 'Training' ]

# setting response variable
y_train = titanic_df[ 'Transported' ][ titanic_df['set']   == 'Training' ]

In [36]:
# Check for missing values in your training data
#print(x_train.isnull().sum())
#print(y_train.isnull().sum())

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Even though there's already a testing set, it's a good idea to build one on the training data to assess model performance and stability qualities.<br>

In [37]:

# developing training and validation sets
x_train_1, x_train_2, y_train_1, y_train_2 = train_test_split(
            x_train,
            y_train.astype(dtype = 'int'),
            random_state = 123,
            test_size    = 0.25,
            stratify     = y_train)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Below is a Logistic Regression model using scikit-learn. Note that as you explore additional scikit-learn models, the only things that need to change are the model name and the model type.<br>

In [38]:
from sklearn.ensemble import HistGradientBoostingClassifier
# # picking a model name
model_name = "Histogram Gradient Boosting Classifier"


# # INSTANTIATING a model object - CHANGE THIS AS NEEDED
model = HistGradientBoostingClassifier(learning_rate=0.11,
                                       max_depth=18,
                                       min_samples_leaf=30,
                                       max_leaf_nodes=31,
                                       loss='auto',
                                       max_iter=100,
                                       n_iter_no_change=200,
                                       max_bins=255,random_state = 219)  

#learning_rate': 0.1, 'max_bins': 255, 'max_depth': 18, 'max_iter': 100, 'max_leaf_nodes': 31, 'min_samples_leaf': 30}                                     
#loss='auto',learning_rate=0.1, max_iter=100, max_leaf_nodes=31, max_depth=15, min_samples_leaf=20,  max_bins=255, warm_start=False,  scoring='loss', verbose=0, random_state = 219

# FITTING to the training data
model_fit = model.fit(x_train_1, y_train_1)


# PREDICTING on the response variable
model_train_pred = model_fit.predict(x_train_1)
model_valid_pred = model_fit.predict(x_train_2)


# SCORING the results (accuracy)
model_train_score = model.score(x_train_1, y_train_1).round(4) # training accuracy
model_valid_score = model.score(x_train_2, y_train_2).round(4) # validation accuracy

# SCORING the results (auc)
model_train_auc = roc_auc_score(y_true  = y_train_1,
                                y_score = model_train_pred).round(decimals = 4)

model_valid_auc = roc_auc_score(y_true  = y_train_2,
                                y_score = model_valid_pred).round(decimals = 4)

# displaying results
print('Training Accuracy:  ', model_train_score)
print('Validation Accuracy:', model_valid_score)
print('Training AUC:       ', model_train_auc)
print('Validation AUC:     ', model_valid_auc)

Training Accuracy:   0.8692
Validation Accuracy: 0.8132
Training AUC:        0.869
Validation AUC:      0.813


<h2>Hyper HistGradientBoostingClassifier</h2>

In [39]:
# from sklearn.ensemble import HistGradientBoostingClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import roc_auc_score

# # Define your parameter grid
# param_grid = {
#     'learning_rate': [0.1, 0.11, 0.12],
#     'max_depth': [15, 18, 20],
#     'min_samples_leaf': [10, 20, 30],
#     'max_leaf_nodes': [31, 40, 50],
#     'max_iter': [100, 200, 300],
#     'max_bins': [128, 255, 512]
# }

# # INSTANTIATING a model object
# model = HistGradientBoostingClassifier(loss='auto', random_state=219)

# # Use GridSearchCV to find the best hyperparameters
# grid_search = GridSearchCV(model, param_grid=param_grid, cv=5)
# grid_search.fit(x_train_1, y_train_1)

# # FITTING to the training data with the best hyperparameters
# model_fit = grid_search.best_estimator_.fit(x_train_1, y_train_1)

# # PREDICTING on the response variable
# model_train_pred = model_fit.predict(x_train_1)
# model_valid_pred = model_fit.predict(x_train_2)

# # SCORING the results (accuracy)
# model_train_score = model_fit.score(x_train_1, y_train_1).round(4) # training accuracy
# model_valid_score = model_fit.score(x_train_2, y_train_2).round(4) # validation accuracy

# # SCORING the results (auc)
# model_train_auc = roc_auc_score(y_true=y_train_1, y_score=model_train_pred).round(decimals=4)
# model_valid_auc = roc_auc_score(y_true=y_train_2, y_score=model_valid_pred).round(decimals=4)

# # displaying results
# print('Training Accuracy:  ', model_train_score)
# print('Validation Accuracy:', model_valid_score)
# print('Training AUC:       ', model_train_auc)
# print('Validation AUC:     ', model_valid_auc)
# print("Best hyperparameters:", grid_search.best_params_)
# print("Best score:", grid_search.best_score_)

<hr style="height:.9px;border:none;color:#333;background-color:#333;" />
<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>

<h2>Creating and Submitting Predictions</h2>
In this competition, your results should be submitted as a .csv file. This file should contain exactly two columns:<br><br>

1. Id (from the test set)
2. SalePrice (predictions from your model)

<br>
The following code with do exactly that. Notice that all we need to do is apply the same X-variables (x_data) to the .predict step to generate our predictions.

In [40]:
# setting x_test
x_test  = titanic_df[ ['CryoSleep_True','Spa','RoomService','VRDeck','ShoppingMall','FoodCourt','HomePlanet_Europa','Deck_B','Deck_C','Side_S','Destination_TRAPPIST-1e','Deck_E','Deck_F','Age','VIP_True','Deck_D','HomePlanet_Mars','Deck_G','Deck_T','Destination_PSO J318.5-22']][ titanic_df['set'] == 'Testing' ]

# PREDICTING on new data
model_pred = model.predict(x_test)

# checking results
model_pred

array([1, 0, 1, ..., 1, 1, 1])

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
From here, we simply create a DataFrame with the original passenger ids (&nbsp;<em>PassengerId</em>&nbsp;) and the predicted values from the model (&nbsp;<em>Transported</em>&nbsp;).<br>

In [41]:
# saving predictions with their respective Ids from the test set
predictions = pd.DataFrame(data = { 'PassengerId' : data_test['PassengerId'],
                                    'Transported' : model_pred.astype(bool)               } )

# checking the results
predictions.head(n = 5)

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [42]:
predictions['Transported'].value_counts(normalize=True)

True     0.52607
False    0.47393
Name: Transported, dtype: float64

<hr style="height:.9px;border:none;color:#333;background-color:#333;" /><br>
Finally, we save the DataFrame as a .csv file and we're ready to submit. This file will be available in the <em>/kaggle/working</em> directory.

In [43]:
# sending predictions to .csv file
predictions.to_csv(path_or_buf = 'HGBclassifier80.csv',
                   index = False)

<br>

~~~

  _    _                            _____          _ _             _ 
 | |  | |                          / ____|        | (_)           | |
 | |__| | __ _ _ __  _ __  _   _  | |     ___   __| |_ _ __   __ _| |
 |  __  |/ _` | '_ \| '_ \| | | | | |    / _ \ / _` | | '_ \ / _` | |
 | |  | | (_| | |_) | |_) | |_| | | |___| (_) | (_| | | | | | (_| |_|
 |_|  |_|\__,_| .__/| .__/ \__, |  \_____\___/ \__,_|_|_| |_|\__, (_)
              | |   | |     __/ |                             __/ |  
              |_|   |_|    |___/                             |___/   

~~~


<br>